<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/gridVeg_groundCover_abundance_matrix_WRANGLE.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Documentation

[Readme](https://docs.google.com/document/d/1JWnhxNjeSQZkSnGhtHP68i_l1mDj4vPFMBdUvGqN0TA/edit#heading=h.b1khpgg2so3y)

# Tools

In [ ]:
install.packages("bigrquery")
library(bigrquery)
library(tidyverse)

Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

also installing the dependencies ‘bit’, ‘bit64’, ‘gargle’, ‘rapidjsonr’


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.2     ✔ purrr   0.3.4
✔ tibble  3.0.3     ✔ dplyr   1.0.0
✔ tidyr   1.1.0     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



# Source

## Database Connection

In [ ]:
# BigQuery API Key
bq_auth(path = "/content/mpg-data-warehouse-api_key-master.json")

In [ ]:
Sys.setenv(BIGQUERY_TEST_PROJECT = "mpg-data-warehouse")

In [ ]:
billing <- bq_test_project()

# Survey Data: Ground Cover

In [ ]:
con_ground <- dbConnect(
  bigrquery::bigquery(),
  project = "mpg-data-warehouse",
  dataset = "vegetation_gridVeg_summaries",
  billing = billing
)

In [ ]:
dbListTables(con_ground)

[1] "gridVeg_groundCover_abundance_matrix"
[2] "gridVeg_groundCover_intercepts"      
[3] "gridVeg_plant_abundance_matrix"      
[4] "gridVeg_plant_binary_matrix"         
[5] "gridVeg_plant_functional_groups"     
[6] "gridVeg_plant_intercepts"            
[7] "gridVeg_species_richness"            
[8] "gridVeg_survey_effort"

In [ ]:
sql_ground <- "SELECT * FROM `mpg-data-warehouse.vegetation_gridVeg_summaries.gridVeg_groundCover_intercepts`"

In [ ]:
bq_ground <- bq_project_query(billing, sql_ground)

In [ ]:
tb_ground <- bq_table_download(bq_ground)

In [ ]:
df_ground <- as.data.frame(tb_ground)

In [ ]:
glimpse(df_ground)

Rows: 22,392
Columns: 7
$ survey_ID             <chr> "234", "234", "234", "234", "234", "236", "236"…
$ year                  <int> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011,…
$ survey_sequence       <chr> "2011-12", "2011-12", "2011-12", "2011-12", "20…
$ grid_point            <int> 48, 48, 48, 48, 48, 295, 295, 295, 295, 295, 20…
$ intercept_ground_code <chr> "SC", "SD", "SE", "SH", "SU", "SC", "SD", "SE",…
$ ground_group          <chr> "dung", "dung", "dung", "dung", "dung", "dung",…
$ intercepts_pct        <dbl> 0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.…


In [ ]:
df_ground %>% 
  distinct(survey_ID) %>% 
  count()

df_ground %>% 
  distinct(year)

n
<int>
1244


year
<int>
2011
2012
2013
2015
2016
2017


In [ ]:
df_ground_select <-
  df_ground %>%
    select(!ground_group)

In [ ]:
df_ground_select %>% 
  

# Wrangle

## Search and Replace

In [ ]:
df_ground_sr <-
  df_ground_select %>% 
    mutate(intercept_ground_code = case_when(
      intercept_ground_code == "BG" ~ "BG",
      intercept_ground_code == "BV" ~ "BV",
      intercept_ground_code == "G" ~ "G",
      intercept_ground_code == "L" ~ "M_L",
      intercept_ground_code == "LIC" ~ "LIC",
      intercept_ground_code == "M" ~ "M_L",
      intercept_ground_code == "M/L" ~ "M_L",
      intercept_ground_code == "OTHER" ~ "OTHER",
      intercept_ground_code == "R" ~ "R",
      intercept_ground_code == "SC" ~ "SC",
      intercept_ground_code == "SD" ~ "SD",
      intercept_ground_code == "SE" ~ "SE",
      intercept_ground_code == "SH" ~ "SH",
      intercept_ground_code == "SU" ~ "SU",
      intercept_ground_code == "WDL" ~ "WDL",
      intercept_ground_code == "WDS" ~ "WDS",
      intercept_ground_code == "WDSTUMP" ~ "WDSTUMP",
      intercept_ground_code == "WDT" ~ "WDT"
    )) %>% glimpse()

Rows: 22,392
Columns: 6
$ survey_ID             <chr> "234", "234", "234", "234", "234", "236", "236"…
$ year                  <int> 2011, 2011, 2011, 2011, 2011, 2011, 2011, 2011,…
$ survey_sequence       <chr> "2011-12", "2011-12", "2011-12", "2011-12", "20…
$ grid_point            <int> 48, 48, 48, 48, 48, 295, 295, 295, 295, 295, 20…
$ intercept_ground_code <chr> "SC", "SD", "SE", "SH", "SU", "SC", "SD", "SE",…
$ intercepts_pct        <dbl> 0.5, 0.0, 0.5, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.…


In [ ]:
# this shows extra 'M_L' because the rows have not been summed
df_ground_sr %>%
  group_by(intercept_ground_code) %>%
  count()

intercept_ground_code,n
<chr>,<int>
BG,1244
BV,1244
G,1244
LIC,1244
M_L,3732
OTHER,1244
R,1244
SC,1244
SD,1244


## Sum
* Sum 'M_L' values for each survey_ID

In [ ]:
df_ground_sum <-
  df_ground_sr %>%
    # split by survey_ID
    group_by(survey_ID, year, survey_sequence, grid_point, intercept_ground_code) %>%
    # sum intercept_ground_code, this should only change 'M_L' rows
    summarize(intercepts_pct = sum(intercepts_pct)) %>%
    # combine
    spread(intercept_ground_code, intercepts_pct) %>%
    replace(is.na(.), 0) %>% 
    ungroup() %>% glimpse()

`summarise()` regrouping output by 'survey_ID', 'year', 'survey_sequence', 'grid_point' (override with `.groups` argument)



Rows: 1,334
Columns: 20
$ survey_ID       <chr> "012C5FAD-2451-41B0-9E2F-432D1ECEB55C", "0133805F-823…
$ year            <int> 2016, 2016, 2017, 2016, 2015, 2015, 2016, 2016, 2015,…
$ survey_sequence <chr> "2016", "2016", "2017", "2016", "2015", "2015", "2016…
$ grid_point      <int> 285, 505, 401, 23, 23, 323, 323, 511, 402, 196, 226, …
$ BG              <dbl> 15.5, 14.5, 16.5, 16.5, 0.0, 0.0, 0.0, 0.0, 19.5, 37.…
$ BV              <dbl> 8.5, 1.0, 12.5, 8.0, 0.0, 5.0, 0.0, 22.0, 8.0, 8.0, 0…
$ G               <dbl> 8.5, 9.5, 3.5, 6.5, 0.0, 3.0, 0.0, 0.0, 0.0, 24.5, 13…
$ LIC             <dbl> 0.0, 2.0, 0.0, 8.5, 0.0, 0.0, 0.0, 0.0, 0.0, 2.5, 0.0…
$ M_L             <dbl> 63.5, 60.5, 61.0, 59.5, 0.0, 91.0, 0.0, 61.0, 71.0, 1…
$ OTHER           <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ R               <dbl> 1.5, 12.0, 0.0, 0.0, 0.0, 1.0, 0.0, 1.0, 0.5, 8.0, 0.…
$ SC              <dbl> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ SD              <dbl> 0.5,

In [ ]:
df_ground_sum %>% 
  distinct(survey_ID) %>% 
  count()

n
<int>
1244


In [ ]:
head(df_ground_sum)

survey_ID,year,survey_sequence,grid_point,BG,BV,G,LIC,M_L,OTHER,R,SC,SD,SE,SH,SU,WDL,WDS,WDSTUMP,WDT
<chr>,<int>,<chr>,<int>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
012C5FAD-2451-41B0-9E2F-432D1ECEB55C,2016,2016,285,15.5,8.5,8.5,0.0,63.5,0,1.5,0,0.5,0.0,0.0,0,0.0,2.0,0,0
0133805F-8237-4190-B125-14E883158664,2016,2016,505,14.5,1.0,9.5,2.0,60.5,0,12.0,0,0.0,0.0,0.0,0,0.5,0.0,0,0
0155DA4F-B744-4B92-BAE3-326ABC3C586F,2017,2017,401,16.5,12.5,3.5,0.0,61.0,0,0.0,0,0.0,0.0,0.5,0,1.0,5.0,0,0
0159CEF8-3C0B-42D1-B961-03DA47A2C274,2016,2016,23,16.5,8.0,6.5,8.5,59.5,0,0.0,0,0.0,0.5,0.0,0,0.0,0.5,0,0
027F5AE0-F90B-4E4D-81D8-E7674D600243,2015,2015,23,0.0,0.0,0.0,0.0,0.0,0,0.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0
027F5AE0-F90B-4E4D-81D8-E7674D600243,2015,2015,323,0.0,5.0,3.0,0.0,91.0,0,1.0,0,0.0,0.0,0.0,0,0.0,0.0,0,0


# Output

In [ ]:
# output pending comment response
# write_csv(df_ground_sum, path = "gridVeg_groundCover_abundance_matrix_WRANGLE.csv")